In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing import sequence, text
from keras.layers import Input, Embedding
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
import string

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to /home/ryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ryan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
stop_words = list(set(stopwords.words('english')))
punctuation = string.punctuation

In [3]:
df = pd.read_csv('../MachineHackElectronicPrice/feature.csv', index_col =False)

In [6]:
df.head()

,Brand,Model_Info,Additional_Description,Locality,City,State,Price,locality_mean,locality_median,locality_sum,...,cityPrice_mean,cityPrice_sum,localityPrice_median,localityPrice_mean,localityPrice_sum,scPrice_median,scPrice_mean,scPrice_sum,Model_Infolen,Additional_Descriptionlen
0,1,name0 name234 64gb space grey,1yesr old mobile number 999two905two99 bill c...,878,8,2,15000,15000.0,15000.0,15000,...,28035.884615,6560397,15000.0,15000.0,15000,18000.0,25613.621212,6761996,30,52
1,1,name87 watch name251 3 38 mm gps name119 name...,one 101009200 3 perfect working condition def...,940,8,2,17000,14000.0,14000.0,28000,...,28035.884615,6560397,17000.0,17000.0,17000,18000.0,25613.621212,6761996,64,89
2,1,name74 drop name87 name0 name1150 32gb 18000 ...,10100000 6s 16 gb storage around 2 yr old acc...,78,8,2,18000,18000.0,18000.0,18000,...,28035.884615,6560397,18000.0,18000.0,18000,18000.0,25613.621212,6761996,79,119
3,1,iphone 6s space grey 32 gb,101005400 phone 011,191,8,2,17500,17500.0,17500.0,17500,...,28035.884615,6560397,17500.0,17500.0,17500,18000.0,25613.621212,6761996,27,20
4,1,excellent condition new phone one hand use 7 ...,101004500 note 5 101004900 exchange,931,8,2,41000,37250.0,31500.0,149000,...,28035.884615,6560397,41000.0,46000.0,138000,18000.0,25613.621212,6761996,55,36


In [7]:
df['Model_Infolen'] = df['Model_Info'].apply(len)
df['Additional_Descriptionlen'] = df['Additional_Description'].apply(len)

In [8]:
df['text'] = df.apply(lambda row: ' '.join ([str(row['Model_Info']), str(row['Additional_Description'])]), 
                                          axis=1)

In [9]:
df['Model_Info']
df['Additional_Description']
df['text'].head()

0     name0 name234 64gb space grey  1yesr old mobi...
1     name87 watch name251 3 38 mm gps name119 name...
2     name74 drop name87 name0 name1150 32gb 18000 ...
3      iphone 6s space grey 32 gb  101005400 phone 011
4     excellent condition new phone one hand use 7 ...
Name: text, dtype: object

In [10]:
df['char_count'] = df['text'].apply(len)
df['word_count'] = df['text'].apply(lambda x: len(x.split()))
df['word_density'] = df['char_count'] / (df['word_count']+1)
# df['punctuation_count'] = df['text'].apply(lambda x: len("".join(_ for _ in x if _ in punctuation))) 
# df['title_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
# df['upper_case_word_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))
# df['stopword_count'] = df['text'].apply(lambda x: len([wrd for wrd in x.split() if wrd.lower() in stop_words]))

In [11]:
df[['char_count', 'word_count', 'word_density']].head(10)

,char_count,word_count,word_density
0,83,12,6.384615
1,154,23,6.416667
2,199,30,6.419355
3,48,9,4.800000
4,92,15,5.750000
5,1536,224,6.826667
6,47,8,5.222222
7,81,17,4.500000
8,110,16,6.470588
9,69,11,5.750000


In [12]:
# def get_polarity(text):
#     try:
#         textblob = TextBlob(unicode(text, 'utf-8'))
#         pol = textblob.sentiment.polarity
#     except:
#         pol = 0.0
#     return pol

# def get_subjectivity(text):
#     try:
#         textblob = TextBlob(unicode(text, 'utf-8'))
#         subj = textblob.sentiment.subjectivity
#     except:
#         subj = 0.0
#     return subj

In [13]:
# df['polarity'] = df['text'].apply(get_polarity)
# df['subjectivity'] = df['text'].apply(get_subjectivity)

In [86]:
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

df['noun_count'] = df['text'].apply(lambda x: pos_check(x, 'noun'))
df['verb_count'] = df['text'].apply(lambda x: pos_check(x, 'verb'))
df['adj_count'] = df['text'].apply(lambda x: pos_check(x, 'adj'))
df['adv_count'] = df['text'].apply(lambda x: pos_check(x, 'adv'))
#df['pron_count'] = df['text'].apply(lambda x: pos_check(x, 'pron'))

In [88]:
df[['noun_count', 'verb_count', 'adj_count', 'adv_count']].head(10)

,noun_count,verb_count,adj_count,adv_count
0,5,0,3,1
1,8,4,5,0
2,10,0,6,6
3,5,0,0,0
4,7,0,3,0
5,86,14,26,4
6,4,0,1,1
7,15,0,1,0
8,6,1,4,2
9,4,1,2,0


In [129]:
# vect = TfidfVectorizer(stop_words='english', use_idf=True, min_df=50, ngram_range=(1,2))
# X = vect.fit_transform(df['Model_Info'])
# print (X.shape)
# print (type(X))

In [130]:
#df['Model_text'] = df.apply(lambda row: ' '.join ([str(row['Model_Info']), str(row['Additional_Description'])]), 
#                                          axis=1)

#model_text = list(df['Model_Info'].values)

# word level tf-idf for model Infoarticle text
vect_word1 = TfidfVectorizer(max_features=500000,analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
vect_word1.fit(df['Model_Info'].values)
tfidf_complete1 = vect_word1.transform(df['Model_Info'].values)


# # word level tf-idf for resource text
# vect_word = TfidfVectorizer(max_features=1000, analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
# vect_word.fit(resource_text)
# tfidf_resource = vect_word.transform(resource_text)

In [136]:
tfidf_complete1.shape

(2326, 10199)

In [137]:
df1 = pd.DataFrame(tfidf_complete1.toarray()) 

In [138]:
# tfidf1 = dict(zip(vect_word1.get_feature_names(), vect_word.idf_))
# tfidf1 = pd.DataFrame(columns=['model_word_tfidf']).from_dict(dict(tfidf1), orient='index')
# tfidf1.columns = ['model_word_tfidf']

In [139]:
# tfidf1.sort_values(by=['model_word_tfidf'], ascending=False)

In [140]:
# tfidf_complete1.shape

In [141]:
# df['article_text'] = df.apply(lambda row: ' '.join ([str(row['Model_Info']), str(row['Additional_Description'])]), 
#                                           axis=1)

#des_text = list(df['Additional_Description'].values)

# word level tf-idf for model Infoarticle text
vect_word = TfidfVectorizer(max_features=500000, analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
vect_word.fit(df['Additional_Description'].values)
tfidf_complete = vect_word.transform(df['Additional_Description'].values)


# # word level tf-idf for resource text
# vect_word = TfidfVectorizer(max_features=1000, analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
# vect_word.fit(resource_text)
# tfidf_resource = vect_word.transform(resource_text)

In [142]:
tfidf = dict(zip(vect_word.get_feature_names(), vect_word.idf_))
tfidf = pd.DataFrame(columns=['des_word_tfidf']).from_dict(dict(tfidf), orient='index')
tfidf.columns = ['des_word_tfidf']


In [153]:
tfidf[tfidf.sort_values(by=['des_word_tfidf'], ascending=False)>5]

(42804, 1)

In [144]:
df2 = pd.DataFrame(tfidf_complete.toarray()) 

In [145]:
# df['article_text'] = df.apply(lambda row: ' '.join ([str(row['Model_Info']), str(row['Additional_Description'])]), 
#                                           axis=1)

# article_text = list(df['article_text'].values)

# # word level tf-idf for model Infoarticle text
# vect_word = TfidfVectorizer(max_features=25000000000, analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
# vect_word.fit(article_text)
# tfidf_complete = vect_word.transform(article_text)


# # # word level tf-idf for resource text
# # vect_word = TfidfVectorizer(max_features=1000, analyzer='word', stop_words='english', ngram_range=(1,3), dtype=np.float32) 
# # vect_word.fit(resource_text)
# # tfidf_resource = vect_word.transform(resource_text)

,0,1,2,3,4,5,6,7,8,9,...,42794,42795,42796,42797,42798,42799,42800,42801,42802,42803
0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.50258,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2322,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2323,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2324,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [149]:
#ndf = pd.concat([df1,df2,df],axis=1) this will suck big time

In [98]:
# tfidf = dict(zip(vect_word.get_feature_names(), vect_word.idf_))
# tfidf = pd.DataFrame(columns=['article_word_tfidf']).from_dict(dict(tfidf), orient='index')
# tfidf.columns = ['article_word_tfidf']

In [99]:
# tfidf.sort_values(by=['article_word_tfidf'], ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,text,char_count,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count,article_text
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,name0 name234 64gb space grey 1yesr old mobi...,83,12,6.384615,5,0,3,1,0,name0 name234 64gb space grey 1yesr old mobi...
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,name87 watch name251 3 38 mm gps name119 name...,154,23,6.416667,8,4,5,0,0,name87 watch name251 3 38 mm gps name119 name...
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,name74 drop name87 name0 name1150 32gb 18000 ...,199,30,6.419355,10,0,6,6,0,name74 drop name87 name0 name1150 32gb 18000 ...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,iphone 6s space grey 32 gb 101005400 phone 011,48,9,4.800000,5,0,0,0,0,iphone 6s space grey 32 gb 101005400 phone 011
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,excellent condition new phone one hand use 7 ...,92,15,5.750000,7,0,3,0,0,excellent condition new phone one hand use 7 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,iphone x name148 silver one year old 1010023...,109,17,6.055556,9,0,2,1,0,iphone x name148 silver one year old 1010023...
2322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,best name0 deals name35 excellent condition ...,135,19,6.750000,7,0,6,1,0,best name0 deals name35 excellent condition ...
2323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,hurry grab best deals iphones 7 8 full kitt ...,109,19,5.450000,9,1,5,0,0,hurry grab best deals iphones 7 8 full kitt ...
2324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,name58 watch series 4 101008400 4a glassy co...,112,15,7.000000,6,0,4,0,0,name58 watch series 4 101008400 4a glassy co...


In [163]:
# create count vectorizer first
cvectorizer = CountVectorizer(min_df=4, max_features=4000, ngram_range=(1,2))
cvz = cvectorizer.fit_transform(df['text'])

# generate topic models using Latent Dirichlet Allocation
lda_model = LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20, random_state=42)
X_topics = lda_model.fit_transform(cvz)

In [164]:
n_top_words = 8
topic_summaries = []

# get topics and topic terms
topic_word = lda_model.components_ 
vocab = cvectorizer.get_feature_names()

for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' | '.join(topic_words)))

Topic 0: ram | camera | gb | 4gb | display | dual | internal | rom
Topic 1: ipad | 1010062600 | location | 1010062600 location | 9791157412 | name54 name0 | call 9791157412 | 9791157412 saravanan
Topic 2: 5s | 16gb | 101004500 | name0 5s | 5s 16gb | 10100000 5s | 10100000 | cards
Topic 3: available | 1010095300 | 1010037800 | 1010095300 1010037800 | 1010095400 | 101008700 10100000 | accepted | 101008700
Topic 4: sealed | pack | sealed pack | new sealed | green | warranty | 10100000 xs | name49
Topic 5: xs | max | name0 | xs max | 10100000 | 64gb | new | 256gb
Topic 6: 1010011300 | please | accepted | 1010095300 | available | 10100102200 | near | call
Topic 7: available | best | models | price | phone | delivery | cash | name0
Topic 8: condition | name0 | 10100000 | box | 32gb | accessories | bill | available
Topic 9: contact | buyers | serious | genuine | serious buyers | 101006600 | 101006600 galaxy | buyers contact
Topic 10: condition | gb | phone | name0 | old | good | 64 | good con

In [154]:
X_topics.shape

(2326, 20)

In [54]:
df.head()

,Brand,Model_Info,Additional_Description,Locality,City,State,Price,locality_mean,locality_median,locality_sum,...,text,char_count,word_count,word_density,noun_count,verb_count,adj_count,adv_count,pron_count,article_text
0,1,name0 name234 64gb space grey,1yesr old mobile number 999two905two99 bill c...,878,8,2,15000,15000.0,15000.0,15000,...,name0 name234 64gb space grey 1yesr old mobi...,83,12,6.384615,5,0,3,1,0,name0 name234 64gb space grey 1yesr old mobi...
1,1,name87 watch name251 3 38 mm gps name119 name...,one 101009200 3 perfect working condition def...,940,8,2,17000,14000.0,14000.0,28000,...,name87 watch name251 3 38 mm gps name119 name...,154,23,6.416667,8,4,5,0,0,name87 watch name251 3 38 mm gps name119 name...
2,1,name74 drop name87 name0 name1150 32gb 18000 ...,10100000 6s 16 gb storage around 2 yr old acc...,78,8,2,18000,18000.0,18000.0,18000,...,name74 drop name87 name0 name1150 32gb 18000 ...,199,30,6.419355,10,0,6,6,0,name74 drop name87 name0 name1150 32gb 18000 ...
3,1,iphone 6s space grey 32 gb,101005400 phone 011,191,8,2,17500,17500.0,17500.0,17500,...,iphone 6s space grey 32 gb 101005400 phone 011,48,9,4.800000,5,0,0,0,0,iphone 6s space grey 32 gb 101005400 phone 011
4,1,excellent condition new phone one hand use 7 ...,101004500 note 5 101004900 exchange,931,8,2,41000,37250.0,31500.0,149000,...,excellent condition new phone one hand use 7 ...,92,15,5.750000,7,0,3,0,0,excellent condition new phone one hand use 7 ...


In [165]:
topicdf= pd.DataFrame(X_topics) 

In [166]:
topicdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.170736,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778,0.073206,0.573994,0.002778,0.002778,0.002778,0.002778,0.137619,0.002778,0.002778,0.002778,0.002778,0.002778,0.002778
1,0.001923,0.001923,0.001923,0.001923,0.001923,0.001923,0.001923,0.058971,0.001923,0.001923,0.269407,0.168992,0.001923,0.001923,0.001923,0.001923,0.144200,0.001923,0.329583,0.001923
2,0.001613,0.001613,0.001613,0.001613,0.001613,0.001613,0.001613,0.001613,0.341049,0.001613,0.001613,0.001613,0.001613,0.629919,0.001613,0.001613,0.001613,0.001613,0.001613,0.001613
3,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.003846,0.124682,0.003846,0.003846,0.003846,0.003846,0.639454,0.003846,0.003846,0.003846,0.003846,0.170479,0.003846
4,0.002174,0.002174,0.411597,0.002174,0.002174,0.002174,0.002174,0.002174,0.002174,0.002174,0.549273,0.002174,0.002174,0.002174,0.002174,0.002174,0.002174,0.002174,0.002174,0.002174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2321,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.056836,0.002381,0.002381,0.850585,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.002381,0.052103,0.002381
2322,0.001786,0.001786,0.001786,0.001786,0.001786,0.255328,0.001786,0.001786,0.295357,0.001786,0.310731,0.001786,0.001786,0.001786,0.001786,0.001786,0.072116,0.001786,0.039683,0.001786
2323,0.002632,0.002632,0.002632,0.002632,0.002632,0.002632,0.002632,0.893561,0.002632,0.002632,0.059070,0.002632,0.002632,0.002632,0.002632,0.002632,0.002632,0.002632,0.002632,0.002632
2324,0.002941,0.002941,0.002941,0.002941,0.002941,0.125974,0.002941,0.002941,0.078822,0.002941,0.002941,0.147014,0.002941,0.179604,0.002941,0.002941,0.424468,0.002941,0.002941,0.002941


In [167]:
topicdf.shape # this may help 

(2326, 20)

In [168]:
#bert sentence embedding

In [182]:
from bert_serving.client import BertClient


In [183]:

bc = BertClient()

KeyboardInterrupt: 

In [ ]:
bc.encode(['First do it', 'then do it right', 'then do it better'])

In [173]:
##named entity recognition

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

from spacy import displacy
from collections import Counter
#nlp = en_core_web_sm.load()

In [177]:
for row in df['Model_Info']:
    doc = nlp(row)
    if
    df['org']=
    else
    df['org']="NA"
    if
    df['prod']
    else
    df['prod']="NA"
    
    
    

 name0 name234 64gb space grey
 name87 watch name251 3 38 mm gps name119 name183 name62 scratch
 name74 drop name87 name0 name1150 32gb 18000 name42 name156 available name2792
 iphone 6s space grey 32 gb
 excellent condition new phone one hand use 7 month old
 name0 xs max silver 256 gb
 name0 7 128gb jet black
 iphone 6 hai mast hai mere ko pesse ki jarurat hai liye bhech rha hu
 name54 iphone 2 months old full original accesiories
 phone x good condition
 name54 name0 7 128gbname54 sealedcodemiwarrantyexchenge option
 phone xs 256 gb gold 1 month old bill box accesory
 brand new iphone 6 32gb box packed
 phone xr 64 gb bill box packed brand new fix price
 name251 3 42 mm gps brand new bill box packed
 august sale new name87 iphone name234 64gb boxed new
 apply phone
 iphone name112 32gb name54 name120 name115 option cod name2594
 closing name269 name0 models name61 name102 delivery
 3 month warranty name0 xs 64gb name103 like new
 maintained new
 iphone 11 name49 64 gb midnight green

 name123 8 64gb brand new seal pack
 name0 7 32gb matte black pristine condition
 name0 6s 32gbbrand new condition
 name0 5se 64gb gold box full kit
 name87 name0 7
 name87 name0 name112
 name0 7 32gb excellent condition
 name0 7 32gb
 hurry grab best deals iphones
 name0 name414 name229 64gb gold colour
 phone 5s 32gb internal
 name0 7 32 gb name248 name42 name54 mobile box full kit
 name87 iphone name414 name229 256gb
 iphone 7128gbbrand new condition
 name0 xs name229 64gb flawless condition
 name2959 new xs 64gb warranty
 name0 7 32gb black
 cod available name0 models brand new sealed pack repace
 name0 6s 64gb grey colour
 name2953 7 plus
 name123 x 64gb silver color
 name0 6 plus sell exchange
 name123 6 128gb name183 color
 iphonese 64 gb
 name0 7 128gb name42 excellent condition
 iphone xs max 64gbbrand new
 name0 xs name229 64gb flawless condition
 name0 xs 64 gb silver variant
 name0 7 128gb rose gold
 used name0 8 good condition
 phone 6s gold 32 gb
 iphone 6128gbexcellent c